In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
%matplotlib inline


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/merged_files_enron123.csv')

In [ ]:
data.head()

,text,label
0,"Subject: what up , , your cam babe\nwhat are y...",spam
1,Subject: want to make more money ?\norder conf...,spam
2,Subject: food for thoughts\n[\njoin now - take...,spam
3,Subject: miningnews . net newsletter - tuesday...,spam
4,Subject: your pharmacy ta\nwould you want chea...,spam


In [ ]:
data.rename(columns = {'text':'text', 'label':'spam'},
                         inplace = True)

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
data['spam'] = le.fit_transform(data['spam'])

In [ ]:
data['spam'].value_counts()

0    12045
1     4496
Name: spam, dtype: int64

In [ ]:
X = data['text']
y = data['spam']

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size=0.2)

In [ ]:
!pip install keras_preprocessing

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from keras_preprocessing.sequence import pad_sequences

In [ ]:
max_words = 1000
max_len = 150
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = pad_sequences(sequences, maxlen=max_len)

In [ ]:
def RNN():
  inputs = Input(name='inputs', shape=[max_len])
  layer = Embedding(max_words, 50, input_length=max_len)(inputs)
  layer = LSTM(64)(layer)
  layer = Dense(256, name='FC1')(layer)
  layer = Activation('relu')(layer)
  layer = Dropout(0.5)(layer)
  layer = Dense(1, name='out_layer')(layer)
  layer = Activation('sigmoid')(layer)
  model = Model(inputs=inputs,outputs=layer)
  return model

In [ ]:
model = RNN()
model.summary()
model.compile(loss='binary_crossentropy', optimizer=RMSprop(), metrics=['accuracy'])

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 150)]             0         
                                                                 
 embedding_1 (Embedding)     (None, 150, 50)           50000     
                                                                 
 lstm_1 (LSTM)               (None, 64)                29440     
                                                                 
 FC1 (Dense)                 (None, 256)               16640     
                                                                 
 activation_2 (Activation)   (None, 256)               0         
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 out_layer (Dense)           (None, 1)                 257 

In [ ]:
model.fit(sequences_matrix, Y_train, batch_size=120, epochs=10, 
          validation_split=0.2, callbacks=[EarlyStopping(monitor='val_loss', min_delta=0.0001)])

Epoch 1/10
89/89 [==============================] - 45s 476ms/step - loss: 0.2606 - accuracy: 0.8946 - val_loss: 0.1049 - val_accuracy: 0.9607
Epoch 2/10
89/89 [==============================] - 27s 303ms/step - loss: 0.0939 - accuracy: 0.9681 - val_loss: 0.1028 - val_accuracy: 0.9596
Epoch 3/10
89/89 [==============================] - 29s 331ms/step - loss: 0.0692 - accuracy: 0.9779 - val_loss: 0.0749 - val_accuracy: 0.9747
Epoch 4/10
89/89 [==============================] - 29s 322ms/step - loss: 0.0596 - accuracy: 0.9794 - val_loss: 0.0942 - val_accuracy: 0.9720


In [ ]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = pad_sequences(test_sequences, maxlen=max_len)

In [ ]:
accuracy = model.evaluate(test_sequences_matrix, Y_test)

104/104 [==============================] - 2s 23ms/step - loss: 0.0976 - accuracy: 0.9716


In [ ]:
y_pred = model.predict(test_sequences_matrix)

104/104 [==============================] - 3s 27ms/step


In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, classification_report

In [ ]:
y_pred

array([[1.3909720e-04],
       [6.0109196e-06],
       [2.6992973e-07],
       ...,
       [9.9772668e-01],
       [5.2677296e-06],
       [9.9935526e-01]], dtype=float32)

In [ ]:
len(y_pred)

3309

In [ ]:
for i in range(len(y_pred)):
  if y_pred[i] > 0.4:
    y_pred[i] = 1

  else:
    y_pred[i] =0


In [ ]:
y_pred = y_pred.T

In [ ]:
print(classification_report(Y_test, y_pred[0]))

              precision    recall  f1-score   support

           0       0.99      0.97      0.98      2435
           1       0.92      0.98      0.95       874

    accuracy                           0.97      3309
   macro avg       0.95      0.97      0.96      3309
weighted avg       0.97      0.97      0.97      3309



In [ ]:
print(confusion_matrix(Y_test, y_pred[0]))


[[2356   79]
 [  18  856]]


In [ ]:
print(accuracy_score(Y_test, y_pred[0]))

0.9706860078573587


In [ ]:
Testing_K = ["""Say, do you remember where you spent your last month's salary/income?

Maybe you feel like keeping a budget and tracking every expense is extremely difficult.

 

We agree - it is not easy, certainly not to do consistently.

 

Luckily, you no longer have to depend on spreadsheets, complex financial tools, your old ledgers, or even your memory to do this anymore. In this day and age of technology, many apps can help you do that."""]

In [ ]:
txts = tok.texts_to_sequences(Testing_K)
txts = pad_sequences(txts, maxlen=max_len)

In [ ]:
preds = model.predict(txts)
print(preds)

1/1 [==============================] - 0s 55ms/step
[[0.9954246]]
